In [84]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import folium
from folium.map import *

Only look at zip code 14213

## City of Buffalo Auctions
Dataset from here: https://www.buffalony.gov/839/Inrem-51-Foreclosure-Listing

In [85]:
auctions = pd.read_excel('../data/raw/Inrem52.xls')
auctions = auctions[auctions['PROPZIP5'] == 14213]
auctions.head()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,INREMNO,INREMSER,SBL,HSENOFR,STREET,PROPZIP5,PROPCLASS,dbo_propclass_DESC1,dbo_fassessment_DESC1,FRONT,DEPTH,TAX DISTRICT,COUNCIL,OWNER1,OWNER2
26,52,176,0994200005012000,140,ALBANY,14213,411,APARTMENT,WEST COR PLYMOUTH,40.0,100.00,2,NI,"SINEX INVESTMENTS, LLC",NaN
27,52,183,0994200002003000,19,ARKANSAS,14213,220,TWO FAMILY DWELLING,134. W PLYMOUTH,30.0,110.00,2,NI,JCM CONSULT INC,NaN
28,52,184,0994300002002000,141,ARKANSAS,14213,220,TWO FAMILY DWELLING,30. E SHERWOOD,30.0,95.00,2,NI,TORRES JOSE,NaN
29,52,210,0994400001030200,96,EIGHTEENTH,14213,220,TWO FAMILY DWELLING,255. NW MASS,30.0,79.44,2,NI,BLOIS DONALD J,BLOIS PAULA J
30,52,239,0993600005014000,471,FERRY WEST,14213,210,ONE FAMILY DWELLING,315. NW MASS,30.0,101.00,2,NI,BOUDREAU MATTHEW J,NaN


In [86]:
auctions.shape

(30, 15)

## 2018-2019 Assessment Roll
Dataset from here: https://data.buffalony.gov/Government/2018-2019-Assessment-Roll/kckn-jafw
Will be used to join in data on lat lon to auction data

In [114]:
assessment = pd.read_csv('../data/raw/2018-2019_Assessment_Roll.csv')
assessment = assessment[assessment['ZIP CODE (5-DIGIT)'] == 14213]
assessment.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SBL,TAX DISTRICT,PRINT KEY,FRONT,DEPTH,PROPERTY CLASS,PROPERTY CLASS DESCRIPTION,PREVIOUS PROPERTY CLASS,OWNER1,OWNER2,...,# OF BATHS,COUNCIL DISTRICT,POLICE DISTRICT,CENSUS TRACT,CENSUS BLOCK GROUP,CENSUS BLOCK,NEIGHBORHOOD,LATITUDE,LONGITUDE,LOCATION
36,1103600004006100,147002,110.36-4-6.1,61.00,100.0,220,TWO FAMILY DWELLING,220.0,SHEPARD JOHN N JR.,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
114,0897700001004000,147007,89.77-1-4,50.00,79.0,210,ONE FAMILY DWELLING,210.0,ROACH CHRISTINE B &,MICHAEL J,...,2.1,NaN,NaN,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
117,0901300001005110,147010,90.13-1-5.11,75.00,85.0,330,COMMERCIAL VACANT LAND,330.0,TLC PROPERTIES INC,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
118,0994400007027000,147002,99.44-7-27,26.00,60.0,311,RESIDENTIAL VACANT LAND,311.0,ZHU JIMMY,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
120,0992800004009000,147007,99.28-4-9,49.67,164.0,281,MULTIPLE RESIDENCES,281.0,LOBUZZETTA WILLIAM,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN


In [88]:
assessment.shape

(6755, 46)

In [90]:
# inner join datasets
auction_df = pd.merge(auctions, assessment, left_on='SBL', right_on='SBL', how='inner')

In [91]:
auction_df.columns

Index(['INREMNO', 'INREMSER', 'SBL', 'HSENOFR', 'STREET_x', 'PROPZIP5',
       'PROPCLASS', 'dbo_propclass_DESC1', 'dbo_fassessment_DESC1', 'FRONT_x',
       'DEPTH_x', 'TAX DISTRICT_x', 'COUNCIL', 'OWNER1_x', 'OWNER2_x',
       'TAX DISTRICT_y', 'PRINT KEY', 'FRONT_y', 'DEPTH_y', 'PROPERTY CLASS',
       'PROPERTY CLASS DESCRIPTION', 'PREVIOUS PROPERTY CLASS', 'OWNER1_y',
       'OWNER2_y', 'MAIL1', 'MAIL2', 'MAIL3', 'MAIL4', 'HOUSE NUMBER',
       'STREET_y', 'ADDRESS', 'CITY', 'STATE', 'ZIP CODE (5-DIGIT)',
       'ZIP CODE (4-DIGIT)', 'DEED BOOK', 'DEED PAGE', 'DEED DATE', 'ROLL',
       'LAND VALUE', 'TOTAL VALUE', 'SALE PRICE', 'YEAR BUILT',
       'TOTAL LIVING AREA', 'OVERALL CONDITION', 'BUILDING STYLE', 'HEAT TYPE',
       'BASEMENT TYPE', '# OF FIREPLACES', '# OF BEDS', '# OF BATHS',
       'COUNCIL DISTRICT', 'POLICE DISTRICT', 'CENSUS TRACT',
       'CENSUS BLOCK GROUP', 'CENSUS BLOCK', 'NEIGHBORHOOD', 'LATITUDE',
       'LONGITUDE', 'LOCATION'],
      dtype='object')

In [92]:
# trimming down columns
auction_df = auction_df[['SBL',
        'LATITUDE',
        'LONGITUDE',
        'LOCATION',
        'ADDRESS',
        'CITY',
        'STATE',
        'ZIP CODE (5-DIGIT)',
        'PROPERTY CLASS',
        'PROPERTY CLASS DESCRIPTION']]
auction_df.head()

,SBL,LATITUDE,LONGITUDE,LOCATION,ADDRESS,CITY,STATE,ZIP CODE (5-DIGIT),PROPERTY CLASS,PROPERTY CLASS DESCRIPTION
0,0994200005012000,42.912904,-78.896460,"(42.912904268809314, -78.89646013824499)",140 ALBANY,BUFFALO,NY,14213.0,411,APARTMENT
1,0994200002003000,42.914436,-78.896781,"(42.91443559149788, -78.89678117358713)",19 ARKANSAS,BUFFALO,NY,14213.0,220,TWO FAMILY DWELLING
2,0994300002002000,42.914125,-78.892076,"(42.91412524026478, -78.89207562510364)",141 ARKANSAS,BUFFALO,NY,14213.0,220,TWO FAMILY DWELLING
3,0994400001030200,42.913702,-78.887061,"(42.91370243672688, -78.88706092717639)",96 EIGHTEENTH,BUFFALO,NY,14213.0,220,TWO FAMILY DWELLING
4,0993600005014000,42.915375,-78.884262,"(42.91537463970857, -78.88426247132199)",471 FERRY WEST,BUFFALO,NY,14213.0,210,ONE FAMILY DWELLING


In [93]:
auction_df['PROPERTY CLASS DESCRIPTION'].unique()

array(['APARTMENT', 'TWO FAMILY DWELLING', 'ONE FAMILY DWELLING',
       'RESIDENTIAL LAND WITH SMALL IMPROVEMENTS',
       'RESIDENTIAL VACANT LAND', 'COMMERCIAL VACANT LAND'], dtype=object)

## PUSH Properties

In [116]:
# push properties any property that assesment data set where MAIL3 column is '217 GRANT ST'
# filtering done in excel prior
push = pd.read_csv('../data/raw/2017-2018_PUSH_Properties.csv')

# SBL info likely inaccurate, possibly due to scientific notation being used at one point
push.head()

,SBL,TAX DISTRICT,PRINT KEY,FRONT,DEPTH,PROPERTY CLASS,PROP CLASS DESCRIPTION,PREVIOUS PROPERTY CLASS,OWNER1,OWNER2,...,# OF BATHS,COUNCIL DISTRICT,POLICE DISTRICT,CENSUS TRACT,CENSUS BLOCK GROUP,CENSUS BLOCK,NEIGHBORHOOD,LATITUDE,LONGITUDE,LOCATION
0,888000000000000,147007,88.83-1-6,25.0,132.0,330,COMMERCIAL VACANT LAND,330,BUFFALO NEIGHBORHOOD,NaN,...,0.0,NIAGARA,District D,171.0,4,4003,Upper West Side,42.922123,-78.891429,"(42.92212334741801, -78.89142871122729)"
1,888000000000000,147007,88.83-6-3,29.0,99.0,311,RESIDENTIAL VACANT LAND,311,BUFFALO NEIGHBORHOOD,NaN,...,0.0,NIAGARA,District D,61.0,2,2001,Upper West Side,42.922001,-78.889562,"(42.92200126681464, -78.88956222187525)"
2,888000000000000,147007,88.83-8-1,32.0,116.0,220,TWO FAMILY DWELLING,220,BUFFALO NEIGHBORHOOD,NaN,...,2.0,NIAGARA,District D,61.0,4,4001,Upper West Side,42.921965,-78.892146,"(42.92196488134693, -78.89214636139904)"
3,993000000000000,147007,99.26-6-15,28.0,150.0,311,RESIDENTIAL VACANT LAND,311,BUFFALO NEIGHBORHOOD,NaN,...,0.0,NIAGARA,District D,61.0,5,5003,Upper West Side,42.917570,-78.893035,"(42.91756996336571, -78.89303465240198)"
4,993000000000000,147007,99.26-6-16,32.0,150.0,311,RESIDENTIAL VACANT LAND,311,BUFFALO NEIGHBORHOOD,NaN,...,0.0,NIAGARA,District D,61.0,5,5003,Upper West Side,42.917512,-78.893042,"(42.91751226768984, -78.8930415224875)"


## Mapping

In [122]:
# focus on lat lon for 14213 zip code
m = folium.Map(
    location=[42.9166, -78.8920],
    tiles='OpenStreetMap',
    zoom_start=14
    )

# potential marker colors, 'green' removed from list to be used for PUSH properties
marker_colors = ['red', 'blue', 'purple', 'orange', 'darkred',
         'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
         'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
         'gray', 'black', 'lightgray']

######################
# plot auctions
# create Feature Groups for property class
prop_classes = {}
for i, prop_class in enumerate(auction_df['PROPERTY CLASS DESCRIPTION'].unique()):
    prop_classes[prop_class] = [FeatureGroup(name=prop_class), marker_colors[i]]

# plot property classes
for index, row in auction_df.iterrows():
    pop_up_string = 'Address {}, Property Class  {}'.format(row['ADDRESS'], row['PROPERTY CLASS DESCRIPTION'])
    popup = folium.Popup(pop_up_string)
    
    marker = folium.Marker([row['LATITUDE'], row['LONGITUDE']], 
                           popup=popup, 
                           icon=folium.Icon(color=prop_classes[row['PROPERTY CLASS DESCRIPTION']][1]))
    
    # add to layers
    prop_classes[row['PROPERTY CLASS DESCRIPTION']][0].add_child(marker, popup) 
        
# add prop classes to map
for _, value in prop_classes.items():
    m.add_child(value[0])
    
######################
# plot PUSH properties
push_locations = FeatureGroup(name='PUSH Locations')

for index, row in push.iterrows():
    pop_up_string = 'Address {}, Property Class  {}'.format(row['ADDRESS'], row['PROP CLASS DESCRIPTION'])
    popup = folium.Popup(pop_up_string)
    
    marker = folium.Marker([row['LATITUDE'], row['LONGITUDE']],
                           popup=popup,
                           icon=folium.Icon(color='green'))
    
    # add to layers
    push_locations.add_child(marker, popup) 
    
m.add_child(push_locations)

folium.LayerControl().add_to(m)

m.save('../data/processed/push.html')
m